# Wind power calculation

In [73]:
from pathlib import Path
import math

import pandas as pd
import numpy as np
from scipy.interpolate import RegularGridInterpolator

In [2]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

In [3]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation.pkl')

## Wind power density (WPD) equation with extra terms
My equations are as follows:  

- `WPD = (1/2) * (air_density) * (area_blades_swept) * (wind_velocity**3) * (turbine_efficiency)`
- `P_net = WPD * turbine_efficiency - P_required`

#### Variable descriptions:
- `WPD` is the wind power density in watts/m^2 (W/m^2)
    - Units also sometimes written as Wm^-2
- Assume constant air density of 1.225 kg/m3
    - considered constant and equal to $1.22\,{\text{ }}kg{m^{ - 3}}$, according to a standard atmosphere from the International Organization for Standardization
- `turbine_efficiency` is what % of the WPD is converted to electrical energy fed into the grid 
    - Betz limit (Cp) is 0.59 or 59%. This is the theoretical maximum efficiency converting wind power density (WPD) to electricity (assuming no other losses)
    - In reality, most turbines extract about 20-50% (fairly large range and depends on turbine's power curve)
    - Assume 45% converting WPD to rotational energy for the generator; here and further reduce this 
    - Turbine generator efficiency converting mechanical energy to electricity is typically around 0.8 to 0.9 (80-90%)
    - Lump this in with Betz limit to create overall efficiency factor
    - If WPD to generator efficiency is 50%, and generator efficiency is 80%, total turbine efficiency can be approximated like this:
        - 0.5 × 0.8 = 0.4 (40% overall efficiency)
- `P_required` is the electricity required by the wind turbine to operate (in watts)
    - "These turbines are equipped with more advanced systems such as yaw systems to orient the turbine towards the wind, advanced control systems, cooling systems for the generator, heating systems for de-icing, and sometimes hydraulic systems. The operational consumption for such turbines typically ranges from 10,000 to 50,000 Watts (10-50 kW) or 10-50 kWh for one hour."
- `P_net` is the total net power in watts fed to the grid

#### Note: a reminder that MW is a unit of _power_ and MWh is a unit of _energy_!
- Power is watts (W)
- Energy is watt-hours (Wh)

-----

### Calculate hourly net energy output in MWh for one turbine

#### Ideas and questions:
- Use an overall turbine efficiency of 40% (rationale explained above)
- Use net nominal power value (Nettonennleistung) to calculate power_required to operate the turbine itself
    - Turbines require around 1-3% of their net nominal power to operate every hour
    - These values from wind turbine dataset are in kW!
    - Let's approximate using 1%
- Cut-in and cut-out wind speeds? 
    - How and where to include the typical turbine 'cut-in wind speed', the minimum wind speed needed for turbine to start running?
        - This is typically 3-5 m/s
    - Should I cap calculated hub height wind speed to 25 m/s? (can turbines actually take advantage of higher speeds?)
    - "These cut-out values reflect the typical range in which wind turbines can produce electrical power. A rated regime is not considered here since the analysis is not focused on a specific turbine wind power generation"
        - This is applicable to my project as well
    - Reference under section Power Control: https://en.wikipedia.org/wiki/Wind_turbine_design#:~:text=A%20wind%20turbine%20must%20produce,power%20has%20to%20be%20limited
    - Note: cut-off speed seems to be speed at which turbine is shut off and produces no power at all?
- Also approximate rated/nominal wind speed at 11.2 m/s
    - This is the wind speed above which the turbine does NOT produce more energy
    - Came up with this value by running validation on mean values of `Nettonennleistung` and `area_blades` and then executing my `hourly_output_in_mwh` function
        - I then observed what mean wind speed gives the closest value to the nominal power output

In [74]:
df['Nettonennleistung'].describe()

count    30642.000000
mean      1986.239546
std       1117.443776
min          0.120000
25%       1300.000000
50%       2000.000000
75%       2530.000000
max       8000.000000
Name: Nettonennleistung, dtype: float64

##### Function that calculates hourly net power output to grid in units MWh

In [223]:
def hourly_output_in_mwh(net_nominal_power, area_swept_by_blades, wind_speed_hub_height, air_density=1.225):
    """
    Calculate the net hourly power output to the grid for a turbine in MWh
    As this is power calculated for one hour, output is MWh
    Note: input variable net_nominal_power is in kW!
    wind_speed_hub_height is calculated using ERA5 wind speed data and in units m/s
    """
    # Wind speed operational range
    # c. 10.8 km/h
    cut_in_speed = 3
    # c. 93.6 km/h
    cut_out_speed = 26
    
    # Typical rated wind speed (aka nominal wind speed)
    rated_wind_speed = 11.2
    
    # If wind speed lies outside the cut-in and cut-out range, return 0 for no energy output
    if wind_speed_hub_height < cut_in_speed or wind_speed_hub_height >= cut_out_speed:
        return 0
    
    # The case where wind speed is above cut-in speed but still under or equal to rated speed
    if wind_speed_hub_height <= rated_wind_speed:
        # Typical overall turbine efficiency
        turbine_efficiency = 0.40

        # Wind power density equation in units W/m^2
        WPD = 0.5 * air_density * area_swept_by_blades * wind_speed_hub_height**3 

        # Estimate power required in watts using net nominal power of turbine in kW multiplied by percentage factor
        power_required = (net_nominal_power * 1_000) * 0.01

        # Net power output in watts after taking into account turbine efficiency and turbine power requirements
        power_net = WPD * turbine_efficiency - power_required

        # Convert watts to MW
        power_net_mwh = power_net / 1_000_000

        # Calculated for one hour so this is energy with units MWh
        return power_net_mwh
    
    # The case where wind speed is above the rated speed; simply return turbine's rated power in MWh
    else:
        net_nominal_power_mwh = net_nominal_power / 1_000
        return net_nominal_power_mwh

##### Test and validate the function for sensible output

In [224]:
# Take mean of input parameters and pair with various wind speeds to observe output/model behaviour
df[['Nettonennleistung', 'area_blades']].mean()

Nettonennleistung    1986.239546
area_blades          5815.457705
dtype: float64

##### Net nominal power is hit around wind speed 11-12 m/s (use 11.2?)
- This seems to be close to the typical 'rated speed' where higher wind speeds don't produce more energy

In [225]:
# Net nominal power in KW
net_nominal_power = 1986.24
# Area in meters
area_swept_by_blades = 5815.5
# Mean wind speed over the hour in m/s
wind_speed_hub_height = 3

In [226]:
# Execute function to calculate electricity (MWh) fed into grid over one hour
hourly_output_mwh = hourly_output_in_mwh(net_nominal_power, area_swept_by_blades, wind_speed_hub_height)
hourly_output_mwh

0.018607132499999998

-----

## Xarray and `scipy.interpolate.RegularGridInterpolator`